In [378]:
import pandas as pd
import numpy as np
import os
import datetime
from itertools import combinations
import statsmodels.formula.api as smf

This notebook is used to clean up and format the raw EIA data in ../data/EIA/electricity_power_monthly

# Functions

In [273]:
def format_table_602(df_602, table_label):
    ''' Formats the raw excel dataframe for Table 6.2.A and Table 6.2.B into a workable dataframe
    ==============================================================================================
    Input:
        df_602: pd.read_excel output of Table 6.2.A/B 
        table_label: either 'A' or 'B'
    ==============================================================================================
    '''
    
    # Fix source labels
    if table_label == 'A':
        
        column_headings_602A = ['State', 'Renewable Sources', 'Renewable Sources', 'Fossil Fuels', 'Fossil Fuels', 
                                'Hydroelectric Pumped Storage', 'Hydroelectric Pumped Storage', 'Energy Storage', 
                                'Energy Storage', 'Nuclear', 'Nuclear', 'Other', 'Other', 'All', 'All']
        column_headings_602A[1:] = [x + ' Cap' for x in column_headings_602A[1:]]
        df_602.columns = column_headings_602A
        
        
    elif table_label == 'B':
        
        column_headings_602B = ['State', 'Wind', 'Wind', 'Solar Photovoltaic', 'Solar Photovoltaic', 'Solar Thermal', 
                                       'Solar Thermal', 'Conventional Hydroelectric', 'Conventional Hydroelectric', 'Biomass', 
                                       'Biomass', 'Geothermal', 'Geothermal', 'Renewable Sources', 'Renewable Sources', 
                                       'Est Dist Solar Photovoltaic', 'Est Dist Solar Photovoltaic', 
                                       'Est Tot Solar Photovoltaic', 'Est Tot Solar Photovoltaic', 'Est Tot Solar', 
                                       'Est Tot Solar']
        column_headings_602B[1:] = [x + ' Cap' for x in column_headings_602B[1:]]
        df_602.columns = column_headings_602B
        df_602 = df_602.iloc[1:,:]
        
    else:
        
        raise ValueError('Unknown table label')

    # Transpose table
    df_602 = df_602.iloc[1:-1,:].T.reset_index()

    # Fix transposed column headings
    column_headings = df_602.iloc[0].copy()
    column_headings[0] = 'Source'
    column_headings[1] = 'Date'
    df_602.columns = column_headings 

    # Drop first row (contains extraneous info)
    df_602 = df_602.iloc[1:,:]
    
    # Melt data frame so it is aggregated as Source/Date/State/Capacity
    df_602 = pd.melt(df_602, id_vars = ['Source', 'Date'], value_vars = df_602.columns[2:])
    df_602.columns = ['Source', 'Date', 'State', 'Capacity']
    
    # Fix NA values
    df_602['Capacity'] = df_602['Capacity'].apply(lambda x: np.nan if x == 'NM' else x) 
    
    return df_602

In [274]:
def format_table_13A(df_13A):
    ''' Formats the raw excel dataframe for Table 1.3.A into a workable dataframe
    ==============================================================================================
    Input:
        df_602: pd.read_excel output of Table 1.3.A
    ==============================================================================================
    '''
    
    # Fix source labels
    df_13A.columns = ['State', 'All Sectors Gen', 'All Sectors Gen', 'Percent Change', 
                      'Electricty Power Sector Utility Gen', 'Electricity Power Sector Utility Gen', 
                      'Electricty Power Sector Independent Gen', 'Electricity Power Sector Independent Gen', 
                      'Commercial Sector Gen', 'Commercial Sector Gen', 
                      'Industrial Sector Gen', 'Industrial Sector Gen']

    # Transpose table
    df_13A = df_13A.drop('Percent Change', axis = 1)
    df_13A = df_13A.iloc[4:-1,:].T.reset_index()

    # Fix transposed column headings
    column_headings = df_13A.iloc[0].copy()
    column_headings[0] = 'Source'
    column_headings[1] = 'Date'
    df_13A.columns = column_headings 

    # Drop first row (contains extraneous info)
    df_13A = df_13A.iloc[1:,:]
    
    # Melt data frame so it is aggregated as Source/Date/State/Capacity
    df_13A = pd.melt(df_13A, id_vars = ['Source', 'Date'], value_vars = df_13A.columns[2:])
    df_13A.columns = ['Source', 'Date', 'State', 'Generation']
    
    # Fix NA values and scale generation to MWh
    df_13A['Generation'] = df_13A['Generation'].apply(lambda x: np.nan if x == 'NM' else x*1e3) 
    
    return df_13A

In [275]:
def format_table_56A(df_56A):
    ''' Formats the raw excel dataframe for Table 5.6.A into a workable dataframe
    ==============================================================================================
    Input:
        df_602: pd.read_excel output of Table 5.6.A
    ==============================================================================================
    '''
    
    # Fix source labels
    df_56A.columns = ['State', 'Residential Price', 'Residential Price', 'Commercial Price', 'Commercial Price', 
                      'Industrial Price', 'Industrial Price', 'Transportation Price', 'Transportation Price',
                      'All Sectors Price', 'All Sectors Price']

    # Transpose table
    #df_56A = df_56A.drop('Percent Change', axis = 1)
    df_56A = df_56A.iloc[2:-1,:].T.reset_index()

    # Fix transposed column headings
    column_headings = df_56A.iloc[0].copy()
    column_headings[0] = 'Type'
    column_headings[1] = 'Date'
    df_56A.columns = column_headings 

    # Drop first row (contains extraneous info)
    df_56A = df_56A.iloc[1:,:]
    
    # Melt data frame so it is aggregated as Source/Date/State/Capacity
    df_56A = pd.melt(df_56A, id_vars = ['Type', 'Date'], value_vars = df_56A.columns[2:])
    df_56A.columns = ['Type', 'Date', 'State', 'Price']
    
    # Fix NA values
    df_56A['Price'] = df_56A['Price'].apply(lambda x: np.nan if x == 'NM' else x) 
    
    return df_56A

# Main

## Import and Prepare Data

In [276]:
folders_monthyear = [x for x in os.walk('../data/EIA/electricity_power_monthly/')][0][1]

### Tables 6.2.A and 6.2.B (Capacity)

In [277]:
# list of Table 6.2 dataframes for each month 
df_602_list = []

# For each month/year (subfolder), import Table 6.2.A and Table 6.2.B data
for subfolder in folders_monthyear:
    
    folder = '../data/EIA/electricity_power_monthly/' + subfolder + '/'
    
    # Import Table 6.2.A and 6.2.B
    df_602A = pd.read_excel('../data/EIA/electricity_power_monthly/' + subfolder + '/Table_6_02_A.xlsx')
    df_602B = pd.read_excel('../data/EIA/electricity_power_monthly/' + subfolder + '/Table_6_02_B.xlsx')
    
    # Format 
    df_602A = format_table_602(df_602A, 'A')
    df_602B = format_table_602(df_602B, 'B')
    
    # Merge and add to list 
    df_602_merged = pd.concat([df_602A, df_602B.query('Source != "Renewable Sources"')])
    df_602_list.append(df_602_merged)

### Table 1.3.A (Net Generation)

In [278]:
# list of Table 1.3.A dataframes for each month 
df_13A_list = []

# For each month/year (subfolder), import Table 1.3.A
for subfolder in folders_monthyear:

    # Import Table 1.3.A
    df_13A = pd.read_excel('../data/EIA/electricity_power_monthly/' + subfolder + '/Table_1_03_A.xlsx')
    
    # Format 
    df_13A = format_table_13A(df_13A)
    
    # Append to list
    df_13A_list.append(df_13A)

### Table 5.6.A (Avg Electricity Price)

In [279]:
# list of Table 5.6.A dataframes for each month 
df_56A_list = []

# For each month/year (subfolder), import Table 5.6.A
for subfolder in folders_monthyear:

    # Import Table 1.3.A
    df_56A = pd.read_excel('../data/EIA/electricity_power_monthly/' + subfolder + '/Table_5_06_A.xlsx')
    
    # Format 
    df_56A = format_table_56A(df_56A)
    
    # Append to list
    df_56A_list.append(df_56A)

### Net Monthly Generation by State/Source

In [280]:
# Import monthly generation tables
df_gen_2015 = pd.read_csv('../data/EIA/electricity_power_monthly/generation_monthly_2015.csv')
df_gen_2016 = pd.read_csv('../data/EIA/electricity_power_monthly/generation_monthly_2016.csv')
df_gen_2017 = pd.read_csv('../data/EIA/electricity_power_monthly/generation_monthly_2017.csv')

# Import dataframe of state abbreviations and state names
df_state_abb = pd.read_csv('../data/keys/state_FIPS.csv')
df_state_abb['State Name'] = df_state_abb['State Name'].apply(lambda x: x.capitalize()) 

# Key of state abbreviations to convert into state names
state_abb_dict = df_state_abb.set_index('State Abbreviation').to_dict()['State Name']

# Combine state generation data for each year
df_gen = pd.concat([df_gen_2015, df_gen_2016, df_gen_2017])

# Select only data for total electric power industry
df_gen = df_gen[df_gen['TYPE OF PRODUCER'] == 'Total Electric Power Industry']

# Add state names and remove abbreviations
df_gen['State'] = df_gen['STATE'].apply(lambda x: state_abb_dict.get(x))

# Convert year and month columns to single date column
df_gen['Date'] = df_gen.apply(lambda x: datetime.date(x.YEAR, x.MONTH, 1).strftime('%B %Y'), axis = 1)

# Merge similar sources
df_gen['Generation'] = df_gen.iloc[:,5]
source_type_dict = {'Coal': 'Fossil Fuel', 'Other Biomass': 'Biomass', 'Wood and Wood Derived Fuels': 'Biomass',
                   'Solar Thermal and Photovoltaic': 'Solar', 'Pumped Storage': 'Hydroelectric Pumped Storage',
                   'Petroleum': 'Fossil Fuel', 'Other Gases': 'Fossil Fuel', 
                    'Hydroelectric Conventional': 'Conventional Hydroelectric'}
df_gen['Source'] = df_gen['ENERGY SOURCE'].apply(lambda x: source_type_dict.get(x, x))
df_gen = df_gen.groupby(['Source', 'Date', 'State', 'Generation']).sum().reset_index()

# Remove commas from generation and convert to float
df_gen['Generation'] = df_gen['Generation'].apply(lambda x: float(x.replace(',', '')))

# Update source names
df_gen['Source'] = df_gen['Source'].apply(lambda x: x + ' Gen')

# Remove old columns
df_gen = df_gen.drop(['YEAR', 'MONTH'], axis = 1)

# Rename existing columns
df_gen.columns = ['Source', 'Date', 'State', 'Generation']

df_gen.head()

,Source,Date,State,Generation
0,Biomass Gen,April 2015,Alabama,258567.0
1,Biomass Gen,April 2015,Alabama,3889.0
2,Biomass Gen,April 2015,Alaska,0.0
3,Biomass Gen,April 2015,Alaska,4240.0
4,Biomass Gen,April 2015,Arizona,4365.0


## Merge Data

In [290]:
# Concatenate data for each month/year
df_602_concat = pd.concat(df_602_list).reset_index(drop = True)
df_13A_concat = pd.concat(df_13A_list).reset_index(drop = True)
df_56A_concat = pd.concat(df_56A_list).reset_index(drop = True)

In [291]:
## Pivot columns so index is State/Date and columns describe the value

# Table 1.3.A
df_13A_pivot = df_13A_concat.groupby(['State', 'Date', 'Source'])['Generation'].first().unstack('Source').reset_index()
df_13A_pivot.columns.name = ''

# Table 5.6.A
df_56A_pivot = df_56A_concat.groupby(['State', 'Date', 'Type'])['Price'].first().unstack('Type').reset_index()
df_56A_pivot.columns.name = ''

# Tables 6.2.A and 6.2.B
df_602_pivot = df_602_concat.groupby(['State', 'Date', 'Source'])['Capacity'].first().unstack('Source').reset_index()
df_602_pivot.columns.name = ''

# Monthly Electricity Generation by Source
df_gen_pivot = df_gen.groupby(['State', 'Date', 'Source'])['Generation'].first().unstack('Source').reset_index()
df_gen_pivot.columns.name = ''

In [309]:
# Merge Table 1.3.A and Tables 6.2.A and 6.2.B
df_merged = (df_602_pivot.merge(df_13A_pivot, on = ['State', 'Date'])
             .merge(df_56A_pivot, on = ['State', 'Date'])
             .merge(df_gen_pivot, on = ['State', 'Date']))

# Fix missing values
df_merged = df_merged.applymap(lambda x: x if x != '--' else np.nan)

# Drop redundant columns
df_merged['All Sectors Gen'] = df_merged['Total Gen']
df_merged = df_merged.drop('Total Gen', axis = 1)

df_merged.head()

,State,Date,All Cap,Biomass Cap,Conventional Hydroelectric Cap,Energy Storage Cap,Est Dist Solar Photovoltaic Cap,Est Tot Solar Cap,Est Tot Solar Photovoltaic Cap,Fossil Fuels Cap,...,Biomass Gen,Conventional Hydroelectric Gen,Fossil Fuel Gen,Geothermal Gen,Hydroelectric Pumped Storage Gen,Natural Gas Gen,Nuclear Gen,Other Gen,Solar Gen,Wind Gen
0,Alabama,April 2015,31746.3,615.9,3271.0,0.0,1.9,1.9,1.9,22793.0,...,258567.0,1161149.0,1977.0,NaN,NaN,3532999.0,2687335.0,0.0,NaN,NaN
1,Alabama,April 2016,30157.4,660.5,3271.0,0.0,1.9,1.9,1.9,21159.5,...,237408.0,598864.0,1139.0,NaN,NaN,3701412.0,3223322.0,0.0,NaN,NaN
2,Alabama,August 2015,31746.3,615.9,3271.0,0.0,1.9,1.9,1.9,22793.0,...,285657.0,498923.0,4281.0,NaN,NaN,5233445.0,3697007.0,0.0,NaN,NaN
3,Alabama,August 2016,29220.8,668.9,3271.0,0.0,2.3,2.3,2.3,20214.5,...,282742.0,439382.0,2769.0,NaN,NaN,6017841.0,3611119.0,0.0,NaN,NaN
4,Alabama,December 2015,31281.3,615.9,3271.0,0.0,1.9,1.9,1.9,22328.0,...,2994.0,1473132.0,2488398.0,NaN,NaN,4692349.0,3570193.0,0.0,NaN,NaN


In [310]:
df_merged.columns

Index(['State', 'Date', 'All Cap', 'Biomass Cap',
       'Conventional Hydroelectric Cap', 'Energy Storage Cap',
       'Est Dist Solar Photovoltaic Cap', 'Est Tot Solar Cap',
       'Est Tot Solar Photovoltaic Cap', 'Fossil Fuels Cap', 'Geothermal Cap',
       'Hydroelectric Pumped Storage Cap', 'Nuclear Cap', 'Other Cap',
       'Renewable Sources Cap', 'Solar Photovoltaic Cap', 'Solar Thermal Cap',
       'Wind Cap', 'All Sectors Gen', 'Commercial Sector Gen',
       'Electricity Power Sector Independent Gen',
       'Electricity Power Sector Utility Gen',
       'Electricty Power Sector Independent Gen',
       'Electricty Power Sector Utility Gen', 'Industrial Sector Gen',
       'All Sectors Price', 'Commercial Price', 'Industrial Price',
       'Residential Price', 'Transportation Price', 'Biomass Gen',
       'Conventional Hydroelectric Gen', 'Fossil Fuel Gen', 'Geothermal Gen',
       'Hydroelectric Pumped Storage Gen', 'Natural Gas Gen', 'Nuclear Gen',
       'Other Gen', 'So

In [311]:
df_merged.to_csv('../data/EIA/monthly_state_capacity_generation.csv', index = False)

### Clean 

In [406]:
df_merged_clean = df_merged.copy()

In [407]:
# Select only states (filter out regions)
states_list = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
  "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]

df_merged_clean = df_merged_clean.query('State in @states_list').copy()

In [408]:
# Replace spaces in column names with underscores
df_merged_clean.columns = [col.replace(' ', '_').lower() for col in df_merged_clean.columns]

# Shorten column names
df_merged_clean.columns = [x.replace('conventional_hydroelectric', 'hydro') for x in df_merged_clean.columns]
df_merged_clean.columns = [x.replace('est_tot_solar_photovoltaic', 'solar') for x in df_merged_clean.columns]
df_merged_clean.columns = [x.replace('fossil_fuels', 'fossil') for x in df_merged_clean.columns]
df_merged_clean.columns = [x.replace('fossil_fuel', 'fossil') for x in df_merged_clean.columns]

In [409]:
for source in sources:
    
    # OLS fit 
    source_fit = smf.ols('{0}_gen ~ {0}_cap'.format(source), data = df_merged_clean).fit()
    
    # OLS estimate of gen
    source_gen_hat = source_fit.params[0] + df_merged_clean['{0}_cap'.format(source)]*source_fit.params[1]
    
    # Add gen hat to dataframe
    df_merged_clean['{0}_gen_hat'.format(source)] = source_gen_hat

In [413]:
## Add logged variables

sources = ['fossil', 'geothermal', 'hydro', 'solar', 'wind', 'biomass', 'nuclear']

# Logged variables
for source in sources:
    
    # Generation
    df_merged_clean['ln_' + source + '_gen'] = df_merged_clean[source + '_gen'].apply(lambda x: np.log(x+1))
    df_merged_clean['ln_' + source + '_gen'] = df_merged_clean['ln_' + source + '_gen'].fillna(0)
    
    # Generation Hat
    df_merged_clean['ln_' + source + '_gen_hat'] = df_merged_clean[source + '_gen_hat'].apply(lambda x: np.log(x+1))
    df_merged_clean['ln_' + source + '_gen_hat'] = df_merged_clean['ln_' + source + '_gen_hat'].fillna(0)
    
    # Capacity
    df_merged_clean['ln_' + source + '_cap'] = df_merged_clean[source + '_cap'].apply(lambda x: np.log(x+1))
    df_merged_clean['ln_' + source + '_cap'] = df_merged_clean['ln_' + source + '_cap'].fillna(0)
    
    
# Functions of logged variables
for source_a, source_b in combinations(sources, 2):
    
    ## Generation
    # Generation - Product of Logged Pairs
    df_merged_clean['ln_' + source_a + '_' + source_b + '_gen'] = np.multiply(
        df_merged_clean['ln_' + source_a + '_gen'], df_merged_clean['ln_' + source_b + '_gen']) 
    df_merged_clean['ln_' + source_b + '_' + source_a + '_gen'] = df_merged_clean[
        'ln_' + source_a + '_' + source_b + '_gen']
    
    # Generation - Square of Logged Source
    df_merged_clean['ln_' + source_a + '_' + source_a + '_gen'] = np.multiply(
        df_merged_clean['ln_' + source_a + '_gen'], df_merged_clean['ln_' + source_a + '_gen'])
    df_merged_clean['ln_' + source_b + '_' + source_b + '_gen'] = np.multiply(
        df_merged_clean['ln_' + source_b + '_gen'], df_merged_clean['ln_' + source_b + '_gen'])
    
    # Generation - Differrence of Logged Pairs Squared 
    df_merged_clean['g_' + source_a + '_' + source_b + '_gen'] = np.power(np.subtract(
        df_merged_clean['ln_' + source_a + '_gen'], df_merged_clean['ln_' + source_b + '_gen']), 2)
    
    ## Capacity
    # Capacity - Product of Logged Pairs
    df_merged_clean['ln_' + source_a + '_' + source_b + '_cap'] = np.multiply(
        df_merged_clean['ln_' + source_a + '_cap'], df_merged_clean['ln_' + source_b + '_cap']) 
    df_merged_clean['ln_' + source_b + '_' + source_a + '_cap'] = df_merged_clean[
        'ln_' + source_a + '_' + source_b + '_cap']
    
    # Capacity - Square of Logged Source
    df_merged_clean['ln_' + source_a + '_' + source_a + '_cap'] = np.multiply(
        df_merged_clean['ln_' + source_a + '_cap'], df_merged_clean['ln_' + source_a + '_cap'])
    df_merged_clean['ln_' + source_b + '_' + source_b + '_cap'] = np.multiply(
        df_merged_clean['ln_' + source_b + '_cap'], df_merged_clean['ln_' + source_b + '_cap'])
    
    # Capacity - Differrence of Logged Pairs Squared 
    df_merged_clean['g_' + source_a + '_' + source_b + '_cap'] = np.power(np.subtract(
        df_merged_clean['ln_' + source_a + '_cap'], df_merged_clean['ln_' + source_b + '_cap']), 2)
    
    ## Generation Hat
    # Generation Hat - Product of Logged Pairs
    df_merged_clean['ln_' + source_a + '_' + source_b + '_gen_hat'] = np.multiply(
        df_merged_clean['ln_' + source_a + '_gen_hat'], df_merged_clean['ln_' + source_b + '_gen_hat']) 
    df_merged_clean['ln_' + source_b + '_' + source_a + '_gen_hat'] = df_merged_clean[
        'ln_' + source_a + '_' + source_b + '_gen_hat']
    
    # Generation Hat - Square of Logged Source
    df_merged_clean['ln_' + source_a + '_' + source_a + '_gen_hat'] = np.multiply(
        df_merged_clean['ln_' + source_a + '_gen_hat'], df_merged_clean['ln_' + source_a + '_gen_hat'])
    df_merged_clean['ln_' + source_b + '_' + source_b + '_gen_hat'] = np.multiply(
        df_merged_clean['ln_' + source_b + '_gen_hat'], df_merged_clean['ln_' + source_b + '_gen_hat'])
    
    # Generation Hat - Differrence of Logged Pairs Squared 
    df_merged_clean['g_' + source_a + '_' + source_b + '_gen_hat'] = np.power(np.subtract(
        df_merged_clean['ln_' + source_a + '_gen_hat'], df_merged_clean['ln_' + source_b + '_gen_hat']), 2)
    
    
# Other
df_merged_clean['ln_all_sec_pr']  = np.log(df_merged_clean['all_sectors_price'] + 1)
df_merged_clean['ln_all_sec_gen'] = np.log(df_merged_clean['all_sectors_gen']   + 1)

In [414]:
df_merged_clean.head()

,state,date,all_cap,biomass_cap,hydro_cap,energy_storage_cap,est_dist_solar_photovoltaic_cap,est_tot_solar_cap,solar_cap,fossil_cap,...,g_solar_nuclear_cap,ln_wind_biomass_cap,ln_biomass_wind_cap,g_wind_biomass_cap,ln_wind_nuclear_cap,ln_nuclear_wind_cap,g_wind_nuclear_cap,ln_biomass_nuclear_cap,ln_nuclear_biomass_cap,g_biomass_nuclear_cap
0,Alabama,April 2015,31746.3,615.9,3271.0,0.0,1.9,1.9,1.9,22793.0,...,55.739251,0.0,0.0,41.276859,0.0,0.0,72.770849,54.806497,54.806497,4.434715
1,Alabama,April 2016,30157.4,660.5,3271.0,0.0,1.9,1.9,1.9,21159.5,...,55.739251,0.0,0.0,42.178660,0.0,0.0,72.770849,55.401957,55.401957,4.145594
2,Alabama,August 2015,31746.3,615.9,3271.0,0.0,1.9,1.9,1.9,22793.0,...,55.739251,0.0,0.0,41.276859,0.0,0.0,72.770849,54.806497,54.806497,4.434715
3,Alabama,August 2016,29220.8,668.9,3271.0,0.0,2.3,2.3,2.3,20214.5,...,53.826590,0.0,0.0,42.342721,0.0,0.0,72.770849,55.509600,55.509600,4.094369
4,Alabama,December 2015,31281.3,615.9,3271.0,0.0,1.9,1.9,1.9,22328.0,...,55.739251,0.0,0.0,41.276859,0.0,0.0,72.770849,54.806497,54.806497,4.434715


### Export

In [415]:
df_merged_clean.to_csv('../data/processed/gen_cap_reg_data.csv', index = False)